In [44]:
import pandas as pd


In [45]:
df1 = pd.read_csv('train_bodies.csv')
print(df1)
df2= pd.read_csv('train_stances.csv')

print(df2)

      Body ID                                        articleBody
0           0  A small meteorite crashed into a wooded area i...
1           4  Last week we hinted at what was to come as Ebo...
2           5  (NEWSER) – Wonder how long a Quarter Pounder w...
3           6  Posting photos of a gun-toting child online, I...
4           7  At least 25 suspected Boko Haram insurgents we...
...       ...                                                ...
1678     2528  Intelligence agencies hunting for identity of ...
1679     2529  While Daleks "know no fear" and "must not fear...
1680     2530  More than 200 schoolgirls were kidnapped in Ap...
1681     2531  A Guantanamo Bay prisoner released last year a...
1682     2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...

[1683 rows x 2 columns]
                                                Headline  Body ID     Stance
0      Police find mass graves with at least '15 bodi...      712  unrelated
1      Hundreds of Palestinians flee floo

In [46]:
train_stances_id=df2['Body ID']
train_bodies_id=df1['Body ID']
train_data=[]
for each in train_bodies_id:
    if each in train_stances_id:

        target_rows1 = df1[df1['Body ID'] == each]
        target_rows2 = df2[df2['Body ID'] == each]
        for index1, row1 in target_rows1.iterrows():
            for index2, row2 in target_rows2.iterrows():
                train_data.append([row1["Body ID"],row1['articleBody'],row2['Body ID'],row2['Headline'],row2['Stance']])
                

for i in range(20):
    print(train_data[i][0],train_data[i][2],train_data[i][3],train_data[i][4])
                

0 0 Soldier shot, Parliament locked down after gunfire erupts at war memorial unrelated
0 0 Tourist dubbed ‘Spider Man’ after spider burrows under skin for days unrelated
0 0 Luke Somers 'killed in failed rescue attempt in Yemen' unrelated
0 0 BREAKING: Soldier shot at War Memorial in Ottawa unrelated
0 0 Giant 8ft 9in catfish weighing 19 stone caught in Italy is thought to be the biggest ever reeled in with a rod and line unrelated
0 0 Enormous 20-stone catfish caught with fishing rod in Italy after 40-minute boat battle unrelated
0 0 Italian catches huge wels catfish; is it a record? unrelated
0 0 Not coming to a store near you: The pumpkin spice condom unrelated
0 0 One gunman killed in shooting on Parliament Hill in Ottawa, hunt on for other shooters unrelated
0 0 Soldier shot at war memorial in Canada unrelated
0 0 Surreal Photos of Fisherman’s Jaw-Dropping Catch Will Likely Have People Wondering If It’s Real unrelated
0 0 Fisherman lands 19 STONE catfish which could be biggest in

In [70]:
df=pd.DataFrame(train_data,columns=['bodies Body ID','body','Stances Body ID','headline','Stance'])
print(df)



       bodies Body ID                                               body  \
0                   0  A small meteorite crashed into a wooded area i...   
1                   0  A small meteorite crashed into a wooded area i...   
2                   0  A small meteorite crashed into a wooded area i...   
3                   0  A small meteorite crashed into a wooded area i...   
4                   0  A small meteorite crashed into a wooded area i...   
...               ...                                                ...   
49967            2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...   
49968            2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...   
49969            2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...   
49970            2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...   
49971            2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...   

       Stances Body ID                                           headline  \
0         

In [50]:
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [51]:
# Define the functions to compute external features
s1=df.loc[0,'body']
s2=df.loc[0,'headline']
print(df.loc[0,'Stance'])

def ngram_match(s1, s2, n):
    """Compute the number of n-grams matched between two strings"""
    ngrams1 = set([s1[i:i+n] for i in range(len(s1)-n+1)])
    ngrams2 = set([s2[i:i+n] for i in range(len(s2)-n+1)])
    return len(ngrams1.intersection(ngrams2))

def char_ngram_match(s1, s2):
    """Compute the number of character n-grams matched between two strings"""
    match = []
    for n in range(2, 17):
        match .append(ngram_match(s1, s2, n))
    return match
print(char_ngram_match(s1,s2))
print(len(char_ngram_match(s1,s2)))

def word_ngram_match(s1, s2):
    """Compute the number of word n-grams matched between two strings"""
    match = []
    for n in range(2, 7):
        s1_tokens = word_tokenize(s1)
        s2_tokens = word_tokenize(s2)
        s1_ngrams = set([' '.join(s1_tokens[i:i+n]) for i in range(len(s1_tokens)-n+1)])
        s2_ngrams = set([' '.join(s2_tokens[i:i+n]) for i in range(len(s2_tokens)-n+1)])
        match.append(len(s1_ngrams.intersection(s2_ngrams)))
    return match

print(word_ngram_match(s1,s2))
print(len(word_ngram_match(s1,s2)))
def tfidf_score(s1, s2):
    """Compute the weighted TF-IDF score between two strings"""
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf = vectorizer.fit_transform([s1, s2])
    return (tfidf * tfidf.T).A[0,1]
print(tfidf_score(s1,s2))

def sentiment_difference(s1, s2):
    """Compute the sentiment difference between two strings"""
    sid = SentimentIntensityAnalyzer()
    s1_sentiment = sid.polarity_scores(s1)['compound']
    s2_sentiment = sid.polarity_scores(s2)['compound']
    return abs(s1_sentiment - s2_sentiment)

print(sentiment_difference(s1,s2))
external_feature=char_ngram_match(s1,s2)+word_ngram_match(s1,s2)+[tfidf_score(s1,s2)]+[sentiment_difference(s1,s2)]
print(external_feature)


unrelated
[52, 32, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
15
[0, 0, 0, 0, 0]
5
0.0
0.20589999999999997
[52, 32, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.20589999999999997]


In [52]:
import copy
y = copy.deepcopy(df['Stance'])
print(y)
print(set(y))
for index, value in y.iteritems():
    if value=="unrelated":
        y[index]=[0,0,1,0]
    elif value=="disagree":
        y[index]=[0,1,0,0]
    elif value=="discuss":
        y[index]=[1,0,0,0]
    elif value=="agree":
        y[index]=[0,0,0,1]
print(y)

0        unrelated
1        unrelated
2        unrelated
3        unrelated
4        unrelated
           ...    
49967        agree
49968        agree
49969        agree
49970        agree
49971        agree
Name: Stance, Length: 49972, dtype: object
{'discuss', 'disagree', 'unrelated', 'agree'}
0        [0, 0, 1, 0]
1        [0, 0, 1, 0]
2        [0, 0, 1, 0]
3        [0, 0, 1, 0]
4        [0, 0, 1, 0]
             ...     
49967    [0, 0, 0, 1]
49968    [0, 0, 0, 1]
49969    [0, 0, 0, 1]
49970    [0, 0, 0, 1]
49971    [0, 0, 0, 1]
Name: Stance, Length: 49972, dtype: object


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Define the functions to compute external features
def ngram_match(s1, s2, n):
    """Compute the number of n-grams matched between two strings"""
    ngrams1 = set([s1[i:i+n] for i in range(len(s1)-n+1)])
    ngrams2 = set([s2[i:i+n] for i in range(len(s2)-n+1)])
    return len(ngrams1.intersection(ngrams2))

def char_ngram_match(s1, s2):
    """Compute the number of character n-grams matched between two strings"""
    match = []
    for n in range(2, 17):
        match .append(ngram_match(s1, s2, n))
    return match

def word_ngram_match(s1, s2):
    """Compute the number of word n-grams matched between two strings"""
    match = []
    for n in range(2, 7):
        s1_tokens = word_tokenize(s1)
        s2_tokens = word_tokenize(s2)
        s1_ngrams = set([' '.join(s1_tokens[i:i+n]) for i in range(len(s1_tokens)-n+1)])
        s2_ngrams = set([' '.join(s2_tokens[i:i+n]) for i in range(len(s2_tokens)-n+1)])
        match.append(len(s1_ngrams.intersection(s2_ngrams)))
    return match

def tfidf_score(s1, s2):
    """Compute the weighted TF-IDF score between two strings"""
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf = vectorizer.fit_transform([s1, s2])
    return (tfidf * tfidf.T).A[0,1]

def sentiment_difference(s1, s2):
    """Compute the sentiment difference between two strings"""
    sid = SentimentIntensityAnalyzer()
    s1_sentiment = sid.polarity_scores(s1)['compound']
    s2_sentiment = sid.polarity_scores(s2)['compound']
    return abs(s1_sentiment - s2_sentiment)

# Load the data as pandas dataframe
import pandas as pd


# Compute the external features and store in numpy array
external_features =[]
for i, row in df.iterrows():
    s1 = row['headline']
    s2 = row['body']
    external_feature=char_ngram_match(s1,s2)+word_ngram_match(s1,s2)+[tfidf_score(s1,s2)]+[sentiment_difference(s1,s2)]
    if i %1000==0:
        print("num:{}, external_feature:{}".format(i,external_feature))
    external_features.append(external_feature)
    # You can add more features here as needed



In [56]:
    

print(external_features[:20])
print(len(external_features[0]))



[[52, 32, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.20589999999999997], [43, 28, 10, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.8053], [31, 16, 5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.13480000000000003], [31, 18, 6, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.20589999999999997], [69, 64, 29, 14, 6, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0073264573132805115, 0.8053], [58, 39, 15, 6, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.42350000000000004], [33, 19, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 1.1235], [39, 29, 18, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.022997222930638134, 0.8053], [56, 32, 14, 6, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.13480000000000003], [32, 21, 7, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.20589999999999997], [57, 25, 9, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.8053], [50, 32, 16, 11, 7, 5, 3, 2, 1, 

In [57]:
df = pd.DataFrame(data=external_features)
df.to_excel('external_features.xlsx', index=False)

In [59]:
print(y)

0        [0, 0, 1, 0]
1        [0, 0, 1, 0]
2        [0, 0, 1, 0]
3        [0, 0, 1, 0]
4        [0, 0, 1, 0]
             ...     
49967    [0, 0, 0, 1]
49968    [0, 0, 0, 1]
49969    [0, 0, 0, 1]
49970    [0, 0, 0, 1]
49971    [0, 0, 0, 1]
Name: Stance, Length: 49972, dtype: object


In [ ]:
#Define the MLP model and pass the external features as input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=22))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categori_crossentropy', metrics=['accuracy'])

# Train the model using the external features
X = external_features
#y = df['Stance']
model.fit(X, y, epochs=10, batch_size=32)